# Modelo preditivo de interceptação de defesa aérea

**Autor:** Enzo Zanatta  
**Matrícula:** 186665  
**URL do GitHub do projeto:** [https://github.com/Enzolinn/projeto-final-data-science-2025-01](https://github.com/Enzolinn/projeto-final-data-science-2025-01)

---

## 1. Resumo do Projeto

O projeto teve como objetivos principais a criação de um processo de análise da capacidade e eficácia de uma estrutura de defesa aérea, podendo ser de um país ou região de menor escala. Para isso, utilizamos o contexto da guerra russo-ucraniana.

Os passos para se chegar a esse processo de análise foram:
- Aplicação de um modelo matemático probabilístico para a análise da eficácia da defesa aérea ucraniana.
- Desenvolvimento de um modelo para a previsão de interceptação de um ataque iminente.
- Predição e análise dos valores necessários de pontos importantes para atingir uma defesa mais bem-sucedida.

> **Nota:** Apesar do uso de dados de ataques reais, as especificações das defesas aéreas ucranianas (local de posicionamento, estoque de munição e prioridade de alvos) são confidenciais e não puderam ser acessadas. Portanto, foram utilizadas generalizações para tentar chegar a valores coerentes.

---

## 2. Dataset Utilizado

- [Massive Missile Attacks on Ukraine (Kaggle)](https://www.kaggle.com/datasets/piterfm/massive-missile-attacks-on-ukraine)

**Fontes:**
- **Logs de Ataques:** `missile_attacks_daily.csv` — compilação de dados públicos (OSINT) sobre ataques aéreos russos.
- **Especificações de Armas:** `missiles_and_uav.csv`.

### 2.1. Variáveis e Transformações

**Pré-processamento dos dados:**
- **Limpeza de Dados:** Conversão de colunas para tipos de dados corretos (numérico, datetime) e tratamento de valores ausentes.

**Engenharia de Features:**
- **P(track) Ponderado:** Probabilidade de Rastreamento (P(track)) estimada em 0.90, calculada como média ponderada baseada no inventário de sistemas de defesa, atribuindo maior peso aos sistemas da era soviética, mais numerosos.
- **Padronização Geográfica:** A coluna de alvos (`target`) foi limpa e padronizada. Nomes de locais com variações (ex: "kyivska oblast", "kyiv region") foram consolidados em um único padrão (ex: "kyiv") usando um dicionário de mapeamento, crucial para a precisão da análise geográfica e do modelo de ML.
- **Criação da Variável Alvo:** A variável `interception_rate` foi calculada (`destroyed / launched`) para servir como alvo de previsão do modelo de Machine Learning.

---

## 3. Modelos

Dois modelos foram centrais para este projeto:
- Um modelo probabilístico do paper ["A Simple Model for Calculating Ballistic Missile Defense Effectiveness"](https://www.jstor.org/stable/26267438) de Dean A. Wilkening, que forneceu o framework matemático.
- Um modelo preditivo desenvolvido para o projeto.

### 3.1. Modelo Probabilístico

Utilizamos a fórmula de Wilkening de forma inversa para diagnosticar a eficácia atual dos sistemas de defesa.

**Fórmula Base:**  
K_w = P(track) \times (1 - (1 - SSPK)^n)

**Aplicação:**  
Ao calcular a eficácia total histórica (K_w) a partir dos dados e definir as premissas de P(track) e n (número de disparos), isolamos e calculamos o SSKP Implícito (Single-Shot Kill Probability) para cada categoria de arma.

### 3.2. Modelo Preditivo

- **Algoritmo:** `GradientBoostingRegressor` do Scikit-learn, escolhido por capturar relações complexas e interações entre variáveis, além de ser um modelo de ensemble robusto e flexível para relações não-lineares.
- **Objetivo:** Prever a `interception_rate` de um ataque.
- **Features Finais:** O modelo foi treinado com as seguintes variáveis:
  - `launched` (tamanho do ataque)
  - `year`, `month` (fatores temporais)
  - `model`, `weapon_category` (características da arma)
  - `target_standardized` (local do alvo)

---

## 4. Resultados e Conclusões

### 4.1. Análise de Eficácia (SSKP)

- **Hierarquia de Ameaças:**  
  A análise quantificou a dificuldade de interceptar cada ameaça, resultando em um SSPK implícito de:
  - ~9% para mísseis balísticos
  - ~46% para mísseis de cruzeiro
  - ~64% para drones UAV

- **Análise Geográfica:**  
  A eficácia da defesa não é uniforme, com regiões como Kyiv apresentando taxas de interceptação significativamente mais altas do que outras, especialmente aquelas mais próximas da fronteira.

### 4.2. Análise de Cenários

- **Limites da Defesa:**  
  Simulações para atingir um objetivo de defesa de 90% contra mísseis balísticos mostraram que a baixa eficácia atual (SSPK de 9%) exigiria um número impraticável de interceptadores. Para ter sucesso com uma doutrina de 2 disparos por alvo, o SSPK precisaria saltar de 9% para mais de 56%.

- **Insight Estratégico:**  
  O fator limitante de uma defesa quase perfeita é a capacidade de rastreamento (P(track)). Nenhum aumento na quantidade ou qualidade dos interceptadores pode compensar a falha em rastrear um alvo de forma confiável.

### 4.3. Conclusão Final

Este projeto demonstrou a possibilidade de se construir um modelo preditivo com impactos reais no cenário de defesa nacional.



In [105]:
import pandas as pd
import numpy as np
import io
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import OneHotEncoder
import warnings
import kagglehub
import joblib

warnings.filterwarnings('ignore')

print("bibliotecas importadas com sucesso")


bibliotecas importadas com sucesso


In [106]:
try:
    path = kagglehub.dataset_download("piterfm/massive-missile-attacks-on-ukraine")

    print("path to dataset files", path)

    attacks_df = pd.read_csv(f"{path}/missile_attacks_daily.csv")
    weapons_df= pd.read_csv(f"{path}/missiles_and_uav.csv")
    print("arquivo missile_attacks_dailycsv carregado com sucesso")
except FileNotFoundError:
    print("erro verifique o diretorio esta correto")
    exit()



path to dataset files /home/enzo/.cache/kagglehub/datasets/piterfm/massive-missile-attacks-on-ukraine/versions/136
arquivo missile_attacks_dailycsv carregado com sucesso


In [107]:
df = pd.merge(attacks_df, weapons_df[['model', 'category', 'guidance_system']], on='model', how='left')

df['event_date'] = pd.to_datetime(df['time_start'], format='mixed', errors='coerce')
df['year'] = df['event_date'].dt.year
numeric_cols = ['launched', 'destroyed']
for col in numeric_cols:
    df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0)
df['destroyed'] = df.apply(lambda row: min(row['launched'], row['destroyed']), axis=1)

print("\ndados de ataques enriquecidos com detalhes das armas amostra")
print(df[['event_date', 'model', 'category', 'launched', 'destroyed', 'guidance_system']].head())


dados de ataques enriquecidos com detalhes das armas amostra
           event_date             model                category  launched  \
0 2025-06-21 21:00:00    Shahed-136/131                     UAV      47.0   
1 2025-06-21 21:00:00  Iskander-M/KN-23       ballistic missile       2.0   
2 2025-06-21 21:00:00             C-300  surface-to-air missile       1.0   
3 2025-06-20 20:00:00    Shahed-136/131                     UAV     272.0   
4 2025-06-21 02:40:00            Kalibr          cruise missile       4.0   

   destroyed guidance_system  
0       18.0             NaN  
1        0.0             NaN  
2        0.0             NaN  
3      140.0             NaN  
4        3.0             NaN  


In [108]:
# calculo de ptrack ponderado com base no inventario
# ==============================================================================

inventory_data = {
    'system': ['Patriot PAC-3', 'SAMP/T', 'IRIS-T SLM', 'NASAMS', 'S-300', 'Buk-M1', 'Gepard'],
    'quantity': [3, 1, 13, 15, 100, 60, 52],
    'p_track_tier': [0.97, 0.96, 0.95, 0.95, 0.90, 0.90, 0.85]
}
inventory_df = pd.DataFrame(inventory_data)

total_quantity = inventory_df['quantity'].sum()
weighted_sum = (inventory_df['quantity'] * inventory_df['p_track_tier']).sum()
weighted_p_track = weighted_sum / total_quantity

P_TRACK = round(weighted_p_track, 3) #

print(f"o ptrack medio ponderado calculado e {P_TRACK}")
print("este valor sera usado em todos os calculos subsequentes")


o ptrack medio ponderado calculado e 0.896
este valor sera usado em todos os calculos subsequentes


In [ ]:
# analise completa de capacidade 
# ==============================================================================

N_SHOTS_MAPPING = {'ballistic missile': 2, 'cruise missile': 2, 'UAV': 1, 'guided bomb': 1, 'surface-to-air missile': 2, 'Unknown': 1}

def calculate_implied_sspk(launched, destroyed, p_track, n_shots):
    if launched == 0: return np.nan
    kw = destroyed / launched
    base_term = 1 - (kw / p_track)
    if base_term < 0: return 0.0
    sspk = 1 - (base_term)**(1 / n_shots)
    return sspk

# calcular o tamanho de cada onda de ataque agrupando por dia e tipo de arma
attack_waves = df.groupby(['year', 'event_date', 'category'])['launched'].sum().reset_index()

# calcular o tamanho medio das ondas por categoria para a analise geral
avg_attack_size_overall = attack_waves.groupby('category')['launched'].mean().reset_index()
avg_attack_size_overall.rename(columns={'launched': 'W_ataque_medio'}, inplace=True)

# calcular o tamanho medio das ondas por categoria e por ano para a analise anual
avg_attack_size_yearly = attack_waves.groupby(['year', 'category'])['launched'].mean().reset_index()
avg_attack_size_yearly.rename(columns={'launched': 'W_ataque_medio'}, inplace=True)
print("tamanho medio real dos ataques calculado a partir dos dados")


stats_by_category = df.groupby('category').agg(
    launched=('launched', 'sum'),
    destroyed=('destroyed', 'sum')
).reset_index()

stats_by_category = pd.merge(stats_by_category, avg_attack_size_overall, on='category')

stats_by_category['Kw_eficacia_geral'] = np.where(stats_by_category['launched'] > 0, stats_by_category['destroyed'] / stats_by_category['launched'], 0)
stats_by_category['n_shots'] = stats_by_category['category'].str.lower().map(N_SHOTS_MAPPING).fillna(1)
stats_by_category['implied_sspk'] = stats_by_category.apply(
    lambda row: calculate_implied_sspk(row['launched'], row['destroyed'], P_TRACK, row['n_shots']), axis=1
)
stats_by_category['P0_vazamento_zero'] = stats_by_category['Kw_eficacia_geral'] ** stats_by_category['W_ataque_medio']

geral_formatado = stats_by_category.copy()
geral_formatado['Kw_eficacia_geral'] = geral_formatado['Kw_eficacia_geral'].apply(lambda x: f"{x:.1%}")
geral_formatado['implied_sspk'] = geral_formatado['implied_sspk'].apply(lambda x: f"{x:.1%}")
geral_formatado['P0_vazamento_zero'] = geral_formatado['P0_vazamento_zero'].apply(lambda x: f"{x:.2%}")
geral_formatado['W_ataque_medio'] = geral_formatado['W_ataque_medio'].round(1)

print("\nanalise de capacidade geral usando o tamanho medio real de ataque")
print(geral_formatado[['category', 'launched', 'destroyed', 'W_ataque_medio', 'Kw_eficacia_geral', 'implied_sspk', 'P0_vazamento_zero']])


stats_by_year = df.groupby(['year', 'category']).agg(
    launched=('launched', 'sum'),
    destroyed=('destroyed', 'sum')
).reset_index()

stats_by_year = pd.merge(stats_by_year, avg_attack_size_yearly, on=['year', 'category'])

stats_by_year['Kw_eficacia_geral'] = np.where(stats_by_year['launched'] > 0, stats_by_year['destroyed'] / stats_by_year['launched'], 0)
stats_by_year['P0_vazamento_zero'] = stats_by_year['Kw_eficacia_geral'] ** stats_by_year['W_ataque_medio']

p0_pivot_table = stats_by_year.pivot_table(index='category', columns='year', values='P0_vazamento_zero')
p0_pivot_table = p0_pivot_table.applymap(lambda x: f"{x:.2%}" if pd.notnull(x) else "N/A")#type:ignore

print("\nevolucao anual da probabilidade de vazamento zero p0 baseado no tamanho medio de ataque de cada ano")
print(p0_pivot_table)

tamanho medio real dos ataques calculado a partir dos dados

analise de capacidade geral usando o tamanho medio real de ataque
                 category  launched  destroyed  W_ataque_medio  \
0                     UAV   37355.0    23455.0            38.7   
1       ballistic missile     500.0       84.0             2.3   
2          cruise missile    3719.0     2594.0             8.9   
3             guided bomb      14.0        9.0             3.5   
4  surface-to-air missile     427.0        2.0             4.5   

  Kw_eficacia_geral implied_sspk P0_vazamento_zero  
0             62.8%        70.1%             0.00%  
1             16.8%         9.9%             1.55%  
2             69.7%        52.9%             4.02%  
3             64.3%        71.7%            21.30%  
4              0.5%         0.3%             0.00%  

evolucao anual da probabilidade de vazamento zero p0 baseado no tamanho medio de ataque de cada ano
year                      2022    2023   2024   2025
cate

In [111]:
# calculo de requisitos para defesa 
# ==============================================================================

ballistic_stats_row = stats_by_category[stats_by_category['category'] == 'ballistic missile']

if not ballistic_stats_row.empty:
    ballistic_stats = ballistic_stats_row.iloc[0]
    CURRENT_SSPK_BALLISTIC = calculate_implied_sspk(ballistic_stats['launched'], ballistic_stats['destroyed'], P_TRACK, n_shots=2)

    TARGET_P0 = 0.10
    ATTACK_SIZE_W = 10
    
    print(f"analisando um objetivo de {TARGET_P0:.0%} de probabilidade de que nenhum dos {ATTACK_SIZE_W} misseis atinja o alvo")

    def calculate_required_shots(target_p0, attack_size_W, p_track, sspk):
        if sspk >= 1.0: return 1
        k_req = target_p0**(1/attack_size_W)
        
        if k_req >= p_track:
            return float('inf') 
            
        log_term = 1 - (k_req / p_track)
        if log_term <= 0 or (1 - sspk) >= 1: return float('inf')
        
        n = np.log(log_term) / np.log(1 - sspk)
        return np.ceil(n)

    def calculate_required_sspk(target_p0, attack_size_W, p_track, n_shots):
        k_req = target_p0**(1/attack_size_W)
        if k_req >= p_track:
            return float('inf') 
            
        base_term = 1 - (k_req / p_track)
        if base_term < 0: return float('inf')
        
        k = 1 - (1 - base_term)**(1/n_shots)
        return k

    req_n = calculate_required_shots(TARGET_P0, ATTACK_SIZE_W, P_TRACK, CURRENT_SSPK_BALLISTIC)
    print(f"\ncenario 1 para atingir {TARGET_P0:.0%} de defesa com o SSPK atual de {CURRENT_SSPK_BALLISTIC:.1%}")
    if np.isinf(req_n):
        print(f"   => nao e possivel atingir o objetivo com o SSPK atual")
    else:
        print(f"   => seria necessario disparar {int(req_n)} interceptadores por missil")

    
else:
    print("\nnao foram encontrados dados para a categoria ballistic missile para realizar a analise de requisitos")

analisando um objetivo de 10% de probabilidade de que nenhum dos 10 misseis atinja o alvo

cenario 1 para atingir 10% de defesa com o SSPK atual de 9.9%
   => seria necessario disparar 21 interceptadores por missil


In [ ]:
# treinamento 
# ==============================================================================

df_sorted = df.sort_values('event_date').copy()
df_sorted.set_index('event_date', inplace=True)

# calcular as somas de launched e destroyed em uma janela de 30 dias para cada categoria
# o groupby garante que o calculo e feito separadamente para cada tipo de arma
rolling_sums = df_sorted.groupby('category')[['launched', 'destroyed']].rolling(window='30D').sum()

# usar shift1 para evitar vazamento de dados
# isso garante que o calculo para um dia especifico use dados apenas ate o dia anterior
rolling_sums_shifted = rolling_sums.groupby('category').shift(1)

# calcular a taxa de interceptacao da janela deslizante nosso kw de 30 dias
rolling_sums_shifted['kw_rolling_30d'] = np.where(
    rolling_sums_shifted['launched'] > 0,
    rolling_sums_shifted['destroyed'] / rolling_sums_shifted['launched'],
    0
)


rolling_sums_shifted.reset_index(inplace=True)
df = pd.merge(
    df,
    rolling_sums_shifted[['event_date', 'category', 'kw_rolling_30d']],
    on=['event_date', 'category'],
    how='left'
)
df['kw_rolling_30d'] = df['kw_rolling_30d'].fillna(0)


location_mapping = {'kyiv region': 'kyiv', 'kyivska': 'kyiv', 'kyiv and kyivska': 'kyiv', 'kyivska oblast': 'kyiv', 'kyivskyi': 'kyiv', 'kyiv city': 'kyiv', 'kharkiv region': 'kharkiv', 'kharkivska': 'kharkiv', 'odesa region': 'odesa', 'odeska': 'odesa', 'dnipropetrovsk region': 'dnipro', 'dnipropetrovska': 'dnipro', 'lviv region': 'lviv', 'lvivska': 'lviv', 'zaporizhzhia region': 'zaporizhzhia', 'mykolaiv region': 'mykolaiv', 'kherson region': 'kherson', 'poltava region': 'poltava', 'cherkasy region': 'cherkasy', 'khmelnytskyi region': 'khmelnytskyi', 'khmelnytska': 'khmelnytskyi', 'sumy region': 'sumy'}
df['main_target'] = df['target'].astype(str).str.split(',').str[0].str.strip().str.lower()
df['target_standardized'] = df['main_target'].replace(location_mapping)
model_df = df[df['launched'] > 0].copy()
model_df['interception_rate'] = model_df['destroyed'] / model_df['launched']
model_df['month'] = model_df['event_date'].dt.month
target_col = 'interception_rate'
y = model_df[target_col]
X = model_df.drop(columns=[target_col, 'event_date', 'time_start', 'time_end', 'target', 'main_target', 'destroyed', 'carrier', 'guidance_system'])

X_train_df = X[X['year'] < 2025]
X_test_df = X[X['year'] >= 2025]
y_train = y.loc[X_train_df.index]
y_test = y.loc[X_test_df.index]

numeric_features = ['launched', 'year', 'month', 'kw_rolling_30d']
categorical_features = ['model', 'category', 'target_standardized']

for col in categorical_features:
    X_train_df[col] = X_train_df[col].fillna('Unknown').astype(str)
    X_test_df[col] = X_test_df[col].fillna('Unknown').astype(str)

encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)
encoder.fit(X_train_df[categorical_features])
X_train_encoded = encoder.transform(X_train_df[categorical_features])
X_test_encoded = encoder.transform(X_test_df[categorical_features])
X_train_final = np.hstack((X_train_df[numeric_features].values, X_train_encoded)) #type:ignore
X_test_final = np.hstack((X_test_df[numeric_features].values, X_test_encoded)) #type:ignore

if len(X_test_df) > 0 and len(X_train_df) > 0:
    model = GradientBoostingRegressor(n_estimators=100, random_state=42)
    model.fit(X_train_final, y_train)
    predictions = model.predict(X_test_final)
    mae = mean_absolute_error(y_test, predictions)
    print(f"\nmodelo avancado treinado erro absoluto medio MAE nos dados de 2025 {mae:.2%}")
else:
    model = GradientBoostingRegressor(n_estimators=100, random_state=42)
    for col in categorical_features:
        X[col] = X[col].fillna('Unknown').astype(str)
    X_numeric_full = X[numeric_features].values
    X_encoded_full = encoder.fit_transform(X[categorical_features])
    X_final_full = np.hstack((X_numeric_full, X_encoded_full))
    model.fit(X_final_full, y)
    print("\nmodelo avancado treinado com todos os dados disponiveis")

joblib.dump(model, 'defense_model.joblib')
joblib.dump(encoder, 'model_encoder.joblib')
print("modelo e encoder com feature de performance recente salvos em arquivos joblib")

feature kw_rolling_30d criada e adicionada ao modelo

modelo avancado treinado erro absoluto medio MAE nos dados de 2025 10.47%
modelo e encoder com feature de performance recente salvos em arquivos joblib


In [110]:
# predicao de ataque iminente
# ==============================================================================

try:
    loaded_model = joblib.load('defense_model.joblib')
    loaded_encoder = joblib.load('model_encoder.joblib')
    print("modelo e encoder carregados dos arquivos com sucesso")


    imminent_attack_df = pd.read_csv('ataque_iminente.csv')
    print("arquivo ataque_iminentecsv carregado com sucesso")
  

    # calculo da performance recente
    dia_13_fev_2025 = pd.Timestamp('2025-02-13 00:00:00')

    start_date_30d_ago = dia_13_fev_2025 - pd.Timedelta(days=30)
    df['event_date'] = pd.to_datetime(df['event_date'], errors='coerce')
    recent_history_df = df[df['event_date'] >= start_date_30d_ago].copy()
    recent_performance = recent_history_df.groupby('category').agg(
        launched_30d=('launched', 'sum'),
        destroyed_30d=('destroyed', 'sum')
    ).reset_index()
    recent_performance['kw_rolling_30d'] = np.where(
        recent_performance['launched_30d'] > 0,
        recent_performance['destroyed_30d'] / recent_performance['launched_30d'],
        0
    )
    
    # preparacao do dataframe de predicao com logica de fallback inteligente
    print("preparando dados para predicao com fallback inteligente")
    prediction_df = pd.merge(imminent_attack_df, weapons_df[['model', 'category']], on='model', how='left')
    prediction_df.rename(columns={'category': 'category'}, inplace=True)
    
    # merge com a performance recente pode gerar nans
    prediction_df = pd.merge(
        prediction_df,
        recent_performance[['category', 'kw_rolling_30d']],
        on='category',
        how='left'
    )
    
    # merge com a performance geral historica para usar como fallback
    # a variavel stats_by_category foi calculada na celula 4
    overall_performance = stats_by_category[['category', 'Kw_eficacia_geral']]
    prediction_df = pd.merge(prediction_df, overall_performance, on='category', how='left')
    
    # aplicacao do fallback inteligente
    # se kw_rolling_30d for nulo sem ataques recentes usa o Kw_eficacia_geral
    prediction_df['kw_rolling_30d'].fillna(prediction_df['Kw_eficacia_geral'], inplace=True)
    # se mesmo o geral for nulo arma totalmente nova ai sim usa 0
    prediction_df['kw_rolling_30d'].fillna(0, inplace=True)
    # remover a coluna auxiliar que nao e mais necessaria
    prediction_df.drop(columns=['Kw_eficacia_geral'], inplace=True)
    
    # adicionar outras features
    prediction_df['year'] = dia_13_fev_2025.year
    prediction_df['month'] = dia_13_fev_2025.month
    prediction_df['main_target'] = prediction_df['target'].astype(str).str.strip().str.lower()
    prediction_df['target_standardized'] = prediction_df['main_target'].replace(location_mapping)
    
    # pipeline de pre processamento final
    categorical_features = ['model', 'category', 'target_standardized']
    numeric_features = ['launched', 'year', 'month', 'kw_rolling_30d']
    for col in categorical_features:
        prediction_df[col] = prediction_df[col].fillna('Unknown').astype(str)
        
    X_pred_categorical = loaded_encoder.transform(prediction_df[categorical_features])
    X_pred_numeric = prediction_df[numeric_features].values
    X_pred_final = np.hstack((X_pred_numeric, X_pred_categorical))
    
    # fazer e apresentar a predicao
    imminent_predictions = loaded_model.predict(X_pred_final)
    sanitized_predictions = np.clip(imminent_predictions, 0, 1)
    prediction_df['predicted_interception_rate'] = sanitized_predictions
    
    print("\n>>> previsao de resultados para o ataque iminente <<<")
    prediction_df['predicted_interception_rate_pct'] = prediction_df['predicted_interception_rate'].apply(lambda x: f"{x:.1%}")
    print(prediction_df[['model', 'launched', 'target', 'predicted_interception_rate_pct']])

except FileNotFoundError:
    print("\nerro arquivos de modelo nao encontrados execute a celula de treinamento para treinar e salvar o modelo primeiro")

modelo e encoder carregados dos arquivos com sucesso
arquivo ataque_iminentecsv carregado com sucesso
preparando dados para predicao com fallback inteligente

>>> previsao de resultados para o ataque iminente <<<
            model  launched   target predicted_interception_rate_pct
0     X-101/X-555        16     lviv                           77.9%
1  Shahed-136/131        30    odesa                           76.1%
2      Iskander-M         4     kyiv                           78.4%
3      Iskander-M        12     kyiv                           76.8%
4      Iskander-M         2    odesa                           11.9%
5      Iskander-M         2     sumy                           11.9%
6  Shahed-136/131       352     sumy                           73.7%
7            X-22         1  dnipro                             8.4%
